In [ ]:
x, y = separate_data(data, normalised=True, weekdays=True)

x = x.numpy().reshape(-1,1)
y = y.numpy().reshape(-1,1)


def get_xticks(data):
    idss = []; yrs = []
    yr = 1969
    for i in range(20):
        df = data.loc[data.year == yr+i]
        yrs.append(yr+i)
        idss.append(df.index[0])
    return idss, yrs
pos, labels = get_xticks(data)

In [ ]:
# from gpflux.architectures import Config, build_constant_input_dim_deep_gp
# from gpflux.models import DeepGP
num_inducing = 1000

Z = np.linspace(x.min(), x.max(), num_inducing).reshape(-1, 1)

kernel1 = gpflow.kernels.Matern52()
inducing_variable1 = gpflow.inducing_variables.InducingPoints(Z.copy())
gp_layer1 = gpflux.layers.GPLayer(kernel1, inducing_variable1, num_data=len(x), num_latent_gps=y.shape[1])

kernel2 = gpflow.kernels.RBF() 
inducing_variable2 = gpflow.inducing_variables.InducingPoints(Z.copy())
gp_layer2 = gpflux.layers.GPLayer(
    kernel2,
    inducing_variable2,
    num_data=len(x),
    num_latent_gps=y.shape[1],
    mean_function=gpflow.mean_functions.Zero(),
)

kernel3 = gpflow.kernels.RBF() 
inducing_variable3 = gpflow.inducing_variables.InducingPoints(Z.copy())
gp_layer3 = gpflux.layers.GPLayer(
    kernel3,
    inducing_variable3,
    num_data=len(x),
    num_latent_gps=y.shape[1],
    mean_function=gpflow.mean_functions.Zero(),
)

likelihood_layer = gpflux.layers.LikelihoodLayer(gpflow.likelihoods.Gaussian(0.1))
two_layer_dgp = gpflux.models.DeepGP([gp_layer1, gp_layer2], likelihood_layer)

model = two_layer_dgp.as_training_model()
model.compile(tf.optimizers.Adam(0.1))

history = model.fit({"inputs": x, "targets": y}, batch_size=500, epochs=50, verbose=1)
plt.plot(history.history["loss"])

model = two_layer_dgp.as_prediction_model()
output = model(x)
mu = output.f_mean.numpy().squeeze()
var = output.f_var.numpy().squeeze()

plt.figure(figsize=(20,5))
plt.plot(x, y, '.', label='true data', c='black', markersize=3)
plt.plot(x, mu, '-', label='Mean predictive posterior', c='C0', linewidth=3)
vv = 1.96 * np.sqrt(var) 
plt.fill_between(x[:,0], (mu-vv), (mu+vv), alpha=0.4, edgecolor='gray', facecolor='C0')
plt.title('DGP with RBF kernels' )
plt.xticks(pos, labels)
plt.xlabel('Date')
plt.ylabel('Births')
plt.legend()
plt.show()

In [ ]:
# from gpflux.architectures import Config, build_constant_input_dim_deep_gp
# from gpflux.models import DeepGP
x, y = separate_data(data, normalised=True, weekdays=True)

x = x.numpy().reshape(-1,1)
y = y.numpy().reshape(-1,1)


def get_xticks(data):
    idss = []; yrs = []
    yr = 1969
    for i in range(20):
        df = data.loc[data.year == yr+i]
        yrs.append(yr+i)
        idss.append(df.index[0])
    return idss, yrs
pos, labels = get_xticks(data)

num_inducing = 3000

Z = np.linspace(x.min(), x.max(), num_inducing).reshape(-1, 1)

kernel1 = gpflow.kernels.RBF() 
inducing_variable1 = gpflow.inducing_variables.InducingPoints(Z.copy())
gp_layer1 = gpflux.layers.GPLayer(kernel1, inducing_variable1, num_data=len(x), num_latent_gps=y.shape[1])

kernel2 = gpflow.kernels.RBF()
inducing_variable2 = gpflow.inducing_variables.InducingPoints(Z.copy())
gp_layer2 = gpflux.layers.GPLayer(
    kernel2,
    inducing_variable2,
    num_data=len(x),
    num_latent_gps=y.shape[1],
    mean_function=gpflow.mean_functions.Zero(),
)

kernel3 = gpflow.kernels.RBF() 
inducing_variable3 = gpflow.inducing_variables.InducingPoints(Z.copy())
gp_layer3 = gpflux.layers.GPLayer(
    kernel3,
    inducing_variable3,
    num_data=len(x),
    num_latent_gps=y.shape[1],
    mean_function=gpflow.mean_functions.Zero(),
)

likelihood_layer = gpflux.layers.LikelihoodLayer(gpflow.likelihoods.Gaussian(0.1))
two_layer_dgp = gpflux.models.DeepGP([gp_layer1, gp_layer2], likelihood_layer)

model = two_layer_dgp.as_training_model()
model.compile(tf.optimizers.Adam(0.1))

history = model.fit({"inputs": x, "targets": y}, batch_size=500, epochs=50, verbose=0)
plt.plot(history.history["loss"])

model = two_layer_dgp.as_prediction_model()
output = model(x)
mu = output.f_mean.numpy().squeeze()
var = output.f_var.numpy().squeeze()

plt.figure(figsize=(20,5))
plt.plot(x, y, '.', label='true data', c='black', markersize=3)
plt.plot(x, mu, '-', label='Mean predictive posterior', c='C0', linewidth=3)
vv = 1.96 * np.sqrt(var) 
plt.fill_between(x[:,0], (mu-vv), (mu+vv), alpha=0.4, edgecolor='gray', facecolor='C0')
plt.title('DGP with RBF kernels' )
plt.xticks(pos, labels)
plt.xlabel('Date')
plt.ylabel('Births')
plt.legend()
plt.show()